In [8]:
"""
    Importing dependencies
"""

import pymongo
import pprint
import pandas as pd 
import numpy as np 
import pytz
import tweepy

from dateutil import parser
from matplotlib.dates import date2num

In [9]:
""" Pretty print config """
pp = pprint.PrettyPrinter(indent=4)

In [10]:
"""
    Connecting to mongodb
"""

DATABASE = "twitter_database"
USERS_COL = "user_collection"
TWEETS_COL = "tweets_collection"
RETWEETS_COL = "retweets_collection"
REPLIES_COL = "replies_collection"
DISTWISE_COL = "distwise_collection"


client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client[DATABASE]
users_col = mydb[USERS_COL]
tweets_col = mydb[TWEETS_COL]
retweets_col = mydb[RETWEETS_COL]
replies_col = mydb[REPLIES_COL]
distwise_col = mydb[DISTWISE_COL]

In [11]:
""" Fetching district location and zonal data """

districts_df = pd.read_csv("../dataset/district_coordinates.csv")
covid_zones_df = pd.read_csv("../dataset/covid_zones.csv")

print(districts_df.head())
print(covid_zones_df.head())

                 State         District   Latitude  Longitude
0  Andaman and Nicobar  Andaman Islands  12.382571  92.822911
1  Andaman and Nicobar  Nicobar Islands   7.835291  93.511601
2       Andhra Pradesh         Adilabad  19.284514  78.813212
3       Andhra Pradesh        Anantapur  14.312066  77.460158
4       Andhra Pradesh         Chittoor  13.331093  78.927639
   No                  District                        State   Zone
0   1             South Andaman  Andaman and Nicobar Islands    Red
1   2                  Nicobars  Andaman and Nicobar Islands  Green
2   3  North and Middle Andaman  Andaman and Nicobar Islands  Green
3   4                   Kurnool               Andhra Pradesh    Red
4   5                    Guntur               Andhra Pradesh    Red


In [15]:
""" Twitter API credentials """

key1="ZDI7Qh5cvY80NusBSAy0gzsA6"
secret_key1="0eptvusxuOfE2HddWg36IYC7YAUVdtSFSxv1g2ojiiofYMTgJW"

key2= 'O0H9liclRNMQ3NdkKmrQjMjwB'
secret_key2='eDnztQY4r00EvA5ODvIt1yNyE9JgWkqrMscC1LbVwvZ1XyaTu2'

key3= 'tQK6xSEUDC8TFpGtpjLmlY6oC'
secret_key3='qiZirqZGG7j3To6BWeyFwrUeCPcOtdmChUDp3Zp9yFy0OQ1MNl'

def get_twitter_auth_api(api_key, api_secret_key):
    auth = tweepy.OAuthHandler(api_key, api_secret_key)
    api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify= True)
    return api

In [13]:
dist_dict = {} 

for ind in covid_zones_df.index:
    dist_dict[covid_zones_df["District"][ind]] = covid_zones_df["Zone"][ind]

In [14]:
api1 = get_twitter_auth_api(key1, secret_key1)
api2 = get_twitter_auth_api(key2, secret_key2)
api3 = get_twitter_auth_api(key3, secret_key3)

radius = 50
count = 1000
distwise_col.drop()

for ind in districts_df.index:
    api = None
    if districts_df["District"][ind] in dist_dict:
        if ind%3==0:
            api = api1
            
        elif ind%3==1:
            api = api2
            
        else:
            api = api3
            
        geocode = (str(districts_df["Latitude"][ind])+","+str(districts_df["Longitude"][ind])+","+str(radius)+"km")
        public_tweets = tweepy.Cursor(api.search,q="corona",count = 100, geocode=geocode, since="2020-01-01",tweet_mode="extended").items()
        print(districts_df["District"][ind])
        tweet_cnt = 0
        try:
            for tweet in public_tweets:
                tweet._json["District"] = districts_df["District"][ind]
                tweet._json["Zone"] = dist_dict[tweet._json["District"]]
                try:
                    distwise_col.insert_one(tweet._json)
                        
                    tweet_cnt = tweet_cnt+1
                    if tweet_cnt == count:
                        break
                        
                except pymongo.errors.DuplicateKeyError:
                    print("Tweet already saved")
                    
            if ind%10==0:
                        print("Tweets saved for district no. ", ind)
        except Exception as e:
            print(e)
            print("Stopped fetching at district no.", ind,",",districts_df["District"][ind])
            break

print("Tweets saved for all districts")

Adilabad
Anantapur
Chittoor
East Godavari
Guntur
Hyderabad
Karimnagar
Khammam
Tweets saved for district no.  10
Krishna
Kurnool
Medak
Nalgonda
Nellore
Nizamabad
Prakasam
Srikakulam
Tweets saved for district no.  20
Vishakhapatnam
Vizianagaram
Warangal
West Godavari
Changlang
East Kameng
East Siang
Kurung Kumey
Lohit
Lower Dibang Valley
Tweets saved for district no.  30
Lower Subansiri
Papum Pare
Tawang
Tirap
Upper Dibang Valley
Upper Siang
Upper Subansiri
West Kameng
West Siang
Barpeta
Tweets saved for district no.  40
Bongaigaon
Cachar
Darrang
Dhemaji
Dibrugarh
Goalpara
Golaghat
Hailakandi
Jorhat
Tweets saved for district no.  50
Kamrup
Karbi Anglong
Karimganj
Kokrajhar
Lakhimpur
Nagaon
Nalbari
Sonitpur
Tinsukia
Araria
Aurangabad
Banka
Begusarai
Bhagalpur
Bhojpur
Buxar
Tweets saved for district no.  70
Darbhanga
Gaya
Gopalganj
Jamui
Jehanabad
Katihar
Khagaria
Kishanganj
Lakhisarai
Madhepura
Tweets saved for district no.  80
Madhubani
Munger
Muzaffarpur
Nalanda
Nawada
Patna
Purnia
Roht

In [17]:
c = 0
for tweets in distwise_col.find()
    c = c+1

print(c)

SyntaxError: invalid syntax (<ipython-input-17-2a2296125c41>, line 2)